In [ ]:
# -*- conding:utf-8 -*-
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path as path
import hyperspy.api as hs
import sys
import sympy as sy
from tqdm import tqdm


In [ ]:
# 画图的初始设置
plt.style.use(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-python\Figure\liuchzzyy.mplstyle')
# display(plt.style.available)
plt.style.use('default')

# 颜色设定
sys.path.append(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Python\Figure')
from colors import tol_cmap, tol_cset
colors = list(tol_cset('vibrant'))
if r'sunset' not in plt.colormaps():
    plt.colormaps.register(tol_cmap('sunset'))
if r'rainbow_PuRd' not in plt.colormaps():
    plt.colormaps.register(tol_cmap('rainbow_PuRd')) # 备用 plasma

# 输出的文件夹
path_out = path(r"C:\Users\chengliu\Desktop\Figure")

In [ ]:
def create_folders(base_path: path, folder_name: str) -> path:
    folder_path = path.joinpath(base_path, folder_name)
    if not folder_path.exists():
        folder_path.mkdir(parents=True, exist_ok=True)
        # print(f"Created folder: {folder_name}")
    return folder_path

def data2map(df: pd.DataFrame, file_name: str, todisplay: bool =False, vmin: float =2, vmax: float =3.5):

    fig = plt.figure(file_name, dpi=100)
    ax = fig.add_subplot()
    p1 = ax.matshow(df.iloc[:, :],
                    cmap='jet',
                    interpolation='nearest',
                    interpolation_stage='rgba',
                    vmin=vmin,
                    vmax=vmax+1e-9,
                    aspect=(0.067/0.032)
                    )
    cbar = plt.colorbar(p1, fraction=0.05, pad=0.01)
    # cbar.ax.set_ylabel('Intensity')
    ax.set_axis_off()

    if todisplay:
        display(df.head(5))
        plt.show()
    return fig

In [ ]:
# to read all master files
path_master_file=list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data\case1_1stCharge_1stDischarge\DownCell_Mn\Mn').glob('*.dat'))
header='Pt_No tripod_z tripod_x energyc a_i0i1_timer a_i0_1 a_i0_2 a_i1_1 a_i1_2 n_timer n_i0_1 n_i0_2 n_i1_1 n_i1_2 n_i2_1 n_i2_2 n_icr n_tcr n_sca1 n_sca2 n_sca3 n_sca4 sr_i_1 dt'
headers=header.split(' ')

In [ ]:
# 处理 mesh 数据
dfout = []
scan_folder_A = create_folders(path_out, '0_PureAbsorption')
# read data files and clean it
for master_file_name in tqdm(path_master_file):
    
    name_folder_A = master_file_name.parts[-1][:-4]

    dfin=pd.read_csv(master_file_name, index_col=None, header=None, names=headers, sep='\s+', comment='#')

    NaNindex=dfin.isnull().any(axis=1).to_numpy().nonzero()[0].tolist()
    dfin.drop(NaNindex, axis=0, inplace=True)

    # 计算 Absorption
    i0=dfin.loc[:,['a_i0_1', 'a_i0_2']].sum(axis=1).values
    i1=dfin.loc[:,['a_i1_1', 'a_i1_2']].sum(axis=1).values
    dfin['absorption']=np.log(i0/i1) # natural base
    df2out = dfin[columns:=['Pt_No','tripod_x','tripod_z','absorption']].copy(deep=True) 

    # 保存每个能量值下的数据
    df2out.to_csv(path.joinpath(scan_folder_A, f"{name_folder_A}_all.csv"), sep=',', index=False)

    dfout.append(df2out)
    
    # 新建对应的文件夹以及获取对应的位置
    # 获取切片的引索序列
    scancuttoff_index_list=df2out[(df2out.loc[:,'Pt_No']==df2out.loc[:,'Pt_No'].min())].index.tolist()
    # scancuttoff_index_list=dfin2[(dfin2.loc[:,'Pt_No']==dfin2.loc[:,'Pt_No'].max())].index.tolist()
    # print(len(scancuttoff_index_list))
    
    # finding global min and max intensity cheapo way for transmission
    vmin = df2out.loc[:,'absorption'].values.min()
    vmax = df2out.loc[:,'absorption'].values.max()
    # print(vmin, vmax)
    
    for i in tqdm(range(len(scancuttoff_index_list)-1)):
        
        scan_name_A=f'{name_folder_A}_{i+1:04d}'
        path_file_folder_A = create_folders(scan_folder_A, name_folder_A)

        a=scancuttoff_index_list[i]
        b=scancuttoff_index_list[i+1]

        # 输出每个能量下的每次扫描数据，并画图
        df2out.iloc[a:b, :].to_csv(path.joinpath(path_file_folder_A, f'{scan_name_A}.csv'), sep=',', index=True, header=True)    
        slice_energy_A = df2out.iloc[a:b, :].pivot(index='tripod_z', columns='tripod_x', values='absorption')
        slice_energy_A.to_csv(path.joinpath(path_file_folder_A,  f'{scan_name_A}_pivot.csv'), sep=',', index=True, header=True) 
        figure = data2map(slice_energy_A, scan_name_A, todisplay=False, vmin=vmin, vmax=vmax)
        figure.savefig(path.joinpath(path_file_folder_A, f'{scan_name_A}.tif'), transparent=False, pad_inches=0.05, bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
        plt.close()

In [ ]:
# ----------- 非线性最小二乘求解函数 -----------
from scipy.optimize import least_squares
def solve_by_nonlinear_least_squares(y_data, E, Ra, Rb):
    def residual(x):
        a, b, k, m, u = x
        res = [a + b - 1, k * E[0] + m - y_data[0]]
        for i in range(1, 5):
            delta_E = E[i] - E[0]
            lhs = a * Ra[i] + b * Rb[i]
            rhs = (y_data[i] - k * delta_E - m) / u
            res.append(lhs - rhs)
        return res

    x0 = [0.5, 0.5, 0.0, 0.0, 1.0]
    bounds = ([0, 0, -np.inf, -np.inf, 1e-6], [1, 1, np.inf, np.inf, np.inf])
    try:
        result = least_squares(residual, x0, bounds=bounds)
        return result.x.tolist() if result.success else None
    except:
        return None

In [ ]:
# Mn 的结果计算
# 文件夹和常量定义

data_folder = create_folders(path_out, r'1_Normalization')

a, b, k, m, u = sy.symbols('a, b, k, m, mu', real=True)
E = [6533.0, 6553.2, 6557.5, 6561.5, 6610.8]
Ra = [0.01, 1.95670, 1.19821, 1.03919, 1.04312]
Rb = [0.01, 0.531024, 1.04128, 1.42149, 0.983545]
Ej_Mn = [0.2265575, 0.5323902]

# 获取子文件夹和 .csv 文件路径
subfolders = sorted([f for f in scan_folder_A.iterdir() if f.is_dir()])
datalist = [list(sub.glob('*[!_pivot].csv')) for sub in subfolders]

# 符号变量定义
a, b, k, m, u = sy.symbols('a b k m mu', real=True)
variables = [a, b, k, m, u]

# 常量
E = [6533.0, 6553.2, 6557.5, 6561.5, 6610.8]
Ra = [0.01, 1.95670, 1.19821, 1.03919, 1.04312]
Rb = [0.01, 0.531024, 1.04128, 1.42149, 0.983545]

sols = []

for i in tqdm(range(len(datalist[0])-4), desc="Processing"):
    dfs = [pd.read_csv(datalist[j][i], sep=',', header=0, comment='#') for j in range(len(datalist))]
    num_rows = len(dfs[0])

    for j in tqdm(range(num_rows)):
        y_data = [dfs[idx].iloc[j, 3] for idx in range(len(datalist))]

        # --- 符号求解 ---
        eqs = [
            sy.Eq(a + b, 1),
            sy.Eq(k * E[0] + m, y_data[0]),
            sy.Eq(a * Ra[1] + b * Rb[1], (y_data[1] - k * (E[1] - E[0]) - m) / u),
            sy.Eq(a * Ra[2] + b * Rb[2], (y_data[2] - k * (E[2] - E[0]) - m) / u),
            sy.Eq(a * Ra[3] + b * Rb[3], (y_data[3] - k * (E[3] - E[0]) - m) / u),
            sy.Eq(a * Ra[4] + b * Rb[4], (y_data[4] - k * (E[4] - E[0]) - m) / u),
        ]

        try:
            sol = sy.solve(eqs, variables, dict=True)
            if sol:
                val = sol[0]
                sols.append([val[a], val[b], val[k], val[m], val[u]])
                continue  # 成功则跳过数值解
        except:
            # --- 非线性最小二乘解 ---
            x_nls = solve_by_nonlinear_least_squares(y_data, E, Ra, Rb)
            sols.append(x_nls if x_nls is not None else [0]*5)
            continue
    # 结果拼接
    result = pd.concat([dfs[i].iloc[:, :3], pd.DataFrame(sols, columns=[r'C(Mn2+)', r'C(MnO2)', 'slope', 'intercept', 'DeltaU'])], axis=1)

    # 标准化浓度
    result[r'C(Mn2+)'] *= Ej_Mn[0]
    result[r'C(MnO2)'] *= Ej_Mn[1]
    result.to_csv(path.joinpath(data_folder, f'{i:04d}_EJ_ratio.csv'), sep=',', index=False, header=True)

    # for ion, ej in zip(['Mn2+', 'MnO2'], Ej_Mn):
    #     pivot_df = result.pivot(index='tripod_z', columns='tripod_x', values=f'C({ion})').astype(float)
    #     pivot_df.to_csv(path.joinpath(data_folder, f'{i:04d}_NormalMn_{ion}_pivot.csv'), sep=',', index=True, header=True)
    #     data2map(pivot_df.T, f'{i}_NormalMn_{ion}', todisplay=False, vmin=-0.03 * ej, vmax=ej)
    #     plt.savefig(path.joinpath(data_folder, f'{i:04d}_NormalMn_{ion}.tif'), bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
    #     plt.close()

In [ ]:
# Mn 的结果计算
# 文件夹和常量定义
data_folder = create_folders(path_out, r'1_Normalization')

a, b, k, m, u = sy.symbols('a, b, k, m, mu', real=True)
E = [6533.0, 6553.2, 6557.5, 6561.5, 6610.8]
Ra = [0.01, 1.95670, 1.19821, 1.03919, 1.04312]
Rb = [0.01, 0.531024, 1.04128, 1.42149, 0.983545]
Ej_Mn = [0.2265575, 0.5323902]

# 获取子文件夹和 .csv 文件路径
subfolders = sorted([f for f in scan_folder_A.iterdir() if f.is_dir()])
datalist = [list(sub.glob('*[!_pivot].csv')) for sub in subfolders]

# 主循环：每组数据文件
for i in tqdm(range(len(datalist[0])), desc="Processing"):
    dfs = [pd.read_csv(datalist[j][i], sep=',', header=0, comment='#') for j in range(len(datalist))]
    sols = []

    # 遍历每个数据点进行符号计算
    for j in range(len(dfs[0])):
        eqs = [
            sy.Eq(a + b, 1),
            sy.Eq(k * E[0] + m, dfs[0].iloc[j, 3]),
            sy.Eq(a * Ra[1] + b * Rb[1], (dfs[1].iloc[j, 3] - k * (E[1] - E[0]) - m) / u),
            sy.Eq(a * Ra[2] + b * Rb[2], (dfs[2].iloc[j, 3] - k * (E[2] - E[0]) - m) / u),
            sy.Eq(a * Ra[3] + b * Rb[3], (dfs[3].iloc[j, 3] - k * (E[3] - E[0]) - m) / u),
            sy.Eq(a * Ra[4] + b * Rb[4], (dfs[4].iloc[j, 3] - k * (E[4] - E[0]) - m) / u),
        ]
        try:
            sol = sy.solve(eqs, [a, b, k, u], dict=True)

            if not sol:
                continue
            else:
                sol = sol[0]  # 获取第一个解
                diff = (sol[a] * Ra[3] + sol[b] * Rb[3]) - (dfs[3].iloc[j, 3] - dfs[0].iloc[j, 3] - sol[k] * (E[3] - E[0])) / sol[u]
                sols.append([sol[a], sol[b], sol[k], sol[u], diff])

        except Exception as e:
            # 如果求解失败或无解，填入 NaN
            sols.append([float('nan')] * len(dfs))

    # 结果拼接
    result = pd.concat([dfs[i].iloc[:, :3], pd.DataFrame(sols, columns=[r'C(Mn2+)', r'C(MnO2)', 'Background_Slope', 'DeltaU', 'Diff(a+b)'])], axis=1)
    result.to_csv(path.joinpath(data_folder, f'{i:04d}_NormalMn_all_ratio.csv'), sep=',', index=False, header=True)

    # 标准化浓度
    result[r'C(Mn2+)'] *= Ej_Mn[0]
    result[r'C(MnO2)'] *= Ej_Mn[1]

    for ion, ej in zip(['Mn2+', 'MnO2'], Ej_Mn):
        pivot_df = result.pivot(index='tripod_z', columns='tripod_x', values=f'C({ion})').astype(float)
        pivot_df.to_csv(path.joinpath(data_folder, f'{i:04d}_NormalMn_{ion}_pivot.csv'), sep=',', index=True, header=True)
        data2map(pivot_df.T, f'{i}_NormalMn_{ion}', todisplay=False, vmin=-0.03 * ej, vmax=ej)
        plt.savefig(path.joinpath(data_folder, f'{i:04d}_NormalMn_{ion}.tif'), bbox_inches='tight', dpi=600, pil_kwargs={"compression": "tiff_lzw"})
        plt.close()

print("Done.")

In [ ]:
# 这里只处理 <Mn - pre edge> 的形式
dfout = dfout
# display(dfout)
pre_edge_folder = create_folders(path_out, 'post-pre')

# caclulations of relative intensity comparing to (pre-edge)
post_pre_Mn = dfout[4]['absorption'] - dfout[0]['absorption'] # post - pre_edge of Mn
Mn2_pre_Mn = dfout[1]['absorption'] - dfout[0]['absorption'] # Mn(II) - pre_edge
Mn3_pre_Mn = dfout[2]['absorption'] - dfout[0]['absorption'] # Mn(III) - pre_edge
Mn4_pre_Mn = dfout[3]['absorption'] - dfout[0]['absorption'] # Mn(IV) - pre_edge

post_pre_Zn = dfout[8]['absorption'] - dfout[5]['absorption'] # post - pre_edge of Zn
ZHS_pre_Zn = dfout[6]['absorption'] - dfout[5]['absorption'] # ZHS - pre_edge
Zn2_pre_Zn = dfout[7]['absorption'] - dfout[5]['absorption'] # Zn(II) - pre_edge

pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], post_pre_Mn], axis = 1).dropna().to_csv(path.join(pre_edge_folder,'post_pre_Mn_all.dat'),index=None, sep= ' ')
pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], Mn2_pre_Mn], axis = 1).dropna().to_csv(path.join(pre_edge_folder,'Mn2_pre_Mn_all.dat'),index=None, sep= ' ')
pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], Mn3_pre_Mn], axis = 1).dropna().to_csv(path.join(pre_edge_folder,'Mn3_pre_Mn_all.dat'),index=None, sep= ' ')
pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], Mn4_pre_Mn], axis = 1).dropna().to_csv(path.join(pre_edge_folder,'Mn4_pre_Mn_all.dat'),index=None, sep= ' ')
pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], post_pre_Zn], axis = 1).dropna().to_csv(path.join(pre_edge_folder,'post_pre_Zn_all.dat'),index=None, sep= ' ')
pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], ZHS_pre_Zn], axis = 1).dropna().to_csv(path.join(pre_edge_folder,'ZHS_pre_Zn_all.dat'),index=None, sep= ' ')
pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], Zn2_pre_Zn], axis = 1).dropna().to_csv(path.join(pre_edge_folder,'Zn2_pre_Zn_all.dat'),index=None, sep= ' ')

Mn_III_IV = Mn3_pre_Mn/Mn4_pre_Mn
Mn_L_S = (2*Mn2_pre_Mn)/(Mn3_pre_Mn+Mn4_pre_Mn)
Zn_ZHS_II = ZHS_pre_Zn/Zn2_pre_Zn

pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], Mn_III_IV], axis = 1).dropna().to_csv(path.join(pre_edge_folder,'Mn_III_IV_all.dat'),index=None, sep= ' ')
pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], Mn_L_S], axis = 1).dropna().to_csv(path.join(pre_edge_folder,'Mn_L_S_all.dat'),index=None, sep= ' ')
pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], Zn_ZHS_II], axis = 1).dropna().to_csv(path.join(pre_edge_folder,'Zn_ZHS_II_all.dat'),index=None, sep= ' ')

# 把每一个合并的文件读取当中的每一小部分，然后画图
Datalist=glob.glob(path.join(pre_edge_folder,file_key))

for Data_file in Datalist:
    # building a folder for each energy file
    name_folder_B =path.split(Data_file)[-1][:-8]
    path_file_folder_B = create_folders(pre_edge_folder, name_folder_B)

    # reading in file and splitting into scans
    dfin2 = pd.read_csv(Data_file, engine = 'c', sep='\s+', index_col = None, header = 0, comment="#")
    scancuttoff_index_list=dfin2[(dfin2.loc[:,'Pt_No']==dfin2.loc[:,'Pt_No'].min())].index.tolist()
    # scancuttoff_index_list=dfin2[(dfin2.loc[:,'Pt_No']==dfin2.loc[:,'Pt_No'].max())].index.tolist()
    # print(dfin2)
    print(len(scancuttoff_index_list))

    # finding global min and max intensity cheapo way for transmission
    vmin = dfin2.loc[:,'absorption'].values.min()
    vmax = dfin2.loc[:,'absorption'].values.max()
    print(vmin,vmax)

    for i in range(len(scancuttoff_index_list)):
        scan_name_B= name_folder_B+'_%s'%"{:04d}".format(i+1)

        if i==0:
            a=0
            b=scancuttoff_index_list[i+1]
        else:
            a=scancuttoff_index_list[i-1]
            b=scancuttoff_index_list[i]
        # if i==0:
        #     a=0
        #     b=scancuttoff_index_list[i]  + 1
        # else:
        #     a=scancuttoff_index_list[i-1] + 1
        #     b=scancuttoff_index_list[i] + 1

        slice_energy_B = dfin2.iloc[a:b, :].pivot(index='tripod_z', columns='tripod_x', values='absorption')
        slice_energy_B.to_csv(path.join(path_file_folder_B,  scan_name_B+'.dat'), sep=' ', index=True, header=True) 
        # slice_energy_B.to_csv(path.join(path_file_folder_B,  scan_name_B+'.dat'), sep=' ', index=None, header=None)
        figure = data2map(slice_energy_B, scan_name_B, todisplay=False, vmin=vmin, vmax=vmax)
        figure.savefig(path.join(path_file_folder_B, scan_name_B+'.tif'), bbox_inches='tight', dpi=600)
        plt.close()

print('Done')

In [ ]:
# 这里只处理 <Mn(II) - pre_edge / post - pre_edge> 的形式
dfout = dfout
# display(dfout)
norm_pre_edge_folder = create_folders(path_out, 'norm_post-pre')

# caclulations of relative intensity comparing to (pre-edge)
post_pre_Mn = dfout[4]['absorption'] - dfout[0]['absorption'] # post - pre_edge of Mn

Mn2_norm_Mn = (dfout[1]['absorption'] - dfout[0]['absorption'])/post_pre_Mn  # Mn(II) - pre_edge / post - pre_edge
Mn3_norm_Mn = (dfout[2]['absorption'] - dfout[0]['absorption'])/post_pre_Mn  # Mn(III) - pre_edge / post - pre_edge
Mn4_norm_Mn = (dfout[3]['absorption'] - dfout[0]['absorption'])/post_pre_Mn  # Mn(IV) - pre_edge / post - pre_edge

post_pre_Zn = dfout[8]['absorption'] - dfout[5]['absorption']  # post - pre_edge of Zn

ZHS_norm_Zn = (dfout[6]['absorption'] - dfout[5]['absorption'])/post_pre_Zn   # ZHS - pre_edge
Zn2_norm_Zn = (dfout[7]['absorption'] - dfout[5]['absorption'])/post_pre_Zn   # Zn(II) - pre_edge

pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], post_pre_Mn], axis = 1).dropna().to_csv(path.join(norm_pre_edge_folder,'post_pre_Mn_all.dat'),index=None, sep= ' ')
pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], Mn2_norm_Mn], axis = 1).dropna().to_csv(path.join(norm_pre_edge_folder,'Mn2_norm_Mn_all.dat'),index=None, sep= ' ')
pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], Mn3_norm_Mn], axis = 1).dropna().to_csv(path.join(norm_pre_edge_folder,'Mn3_norm_Mn_all.dat'),index=None, sep= ' ')
pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], Mn4_norm_Mn], axis = 1).dropna().to_csv(path.join(norm_pre_edge_folder,'Mn4_norm_Mn_all.dat'),index=None, sep= ' ')
pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], post_pre_Zn], axis = 1).dropna().to_csv(path.join(norm_pre_edge_folder,'post_pre_Zn_all.dat'),index=None, sep= ' ')
pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], ZHS_norm_Zn], axis = 1).dropna().to_csv(path.join(norm_pre_edge_folder,'ZHS_norm_Zn_all.dat'),index=None, sep= ' ')
pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], Zn2_norm_Zn], axis = 1).dropna().to_csv(path.join(norm_pre_edge_folder,'Zn2_norm_Zn_all.dat'),index=None, sep= ' ')

norm_Mn_III_IV = Mn3_norm_Mn/Mn4_norm_Mn
norm_Mn_L_S = (2*Mn2_norm_Mn)/(Mn3_norm_Mn+Mn4_norm_Mn)
norm_Zn_ZHS_II = ZHS_norm_Zn/Zn2_norm_Zn

pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], norm_Mn_III_IV], axis = 1).dropna().to_csv(path.join(norm_pre_edge_folder,'norm_Mn_III_IV_all.dat'),index=None, sep= ' ')
pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], norm_Mn_L_S], axis = 1).dropna().to_csv(path.join(norm_pre_edge_folder,'norm_Mn_L_S_all.dat'),index=None, sep= ' ')
pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], norm_Zn_ZHS_II], axis = 1).dropna().to_csv(path.join(norm_pre_edge_folder,'norm_Zn_ZHS_II_all.dat'),index=None, sep= ' ')

# 把每一个合并的文件读取当中的每一小部分，然后画图

Datalist=glob.glob(path.join(norm_pre_edge_folder,file_key))

for Data_file in Datalist:
    # building a folder for each energy file
    name_folder_D =path.split(Data_file)[-1][:-8]
    path_file_folder_D = create_folders(norm_pre_edge_folder, name_folder_D)

    # reading in file and splitting into scans
    dfin2 = pd.read_csv(Data_file, engine = 'c', sep='\s+', index_col = None, header = 0, comment="#")
    scancuttoff_index_list=dfin2[(dfin2.loc[:,'Pt_No']==dfin2.loc[:,'Pt_No'].min())].index.tolist()
    # scancuttoff_index_list=dfin2[(dfin2.loc[:,'Pt_No']==dfin2.loc[:,'Pt_No'].max())].index.tolist()
    # print(dfin2)
    print(len(scancuttoff_index_list))

    # finding global min and max intensity cheapo way for transmission
    vmin = dfin2.loc[:,'absorption'].values.min()
    vmax = dfin2.loc[:,'absorption'].values.max()
    print(vmin,vmax)

    for i in range(len(scancuttoff_index_list)):
        scan_name_D= name_folder_D+'_%s'%"{:04d}".format(i+1)

        if i==0:
            a=0
            b=scancuttoff_index_list[i+1]
        else:
            a=scancuttoff_index_list[i-1]
            b=scancuttoff_index_list[i]
        # if i==0:
        #     a=0
        #     b=scancuttoff_index_list[i]  + 1
        # else:
        #     a=scancuttoff_index_list[i-1] + 1
        #     b=scancuttoff_index_list[i] + 1

        slice_energy_D = dfin2.iloc[a:b, :].pivot(index='tripod_z', columns='tripod_x', values='absorption')
        slice_energy_D.to_csv(path.join(path_file_folder_D,  scan_name_D+'.dat'), sep=' ', index=True, header=True) 
        # slice_energy_B.to_csv(path.join(path_file_folder_D,  scan_name_D+'.dat'), sep=' ', index=None, header=None)
        figure = data2map(slice_energy_D, scan_name_D, todisplay=False, vmin=vmin, vmax=vmax)
        figure.savefig(path.join(path_file_folder_D, scan_name_D+'.tif'), bbox_inches='tight', dpi=600)
        plt.close()

print('Done')

In [ ]:
# 这里只处理 <Mn(II) - pre_edge / post - pre_edge> 的形式，而且扣除掉一些极端点
dfout = dfout
# display(dfout)
norm_pre_edge_folder_E = create_folders(path_out, 'norm_post-pre_E')

# caclulations of relative intensity comparing to (post-edge - pre-edge)
post_pre_Mn = dfout[4]['absorption'] - dfout[0]['absorption']  # post - pre_edge of Mn
Mn2_pre_Mn = dfout[1]['absorption'] - dfout[0]['absorption']   # Mn(II) - pre_edge
Mn3_pre_Mn = dfout[2]['absorption'] - dfout[0]['absorption']   # Mn(III) - pre_edge
Mn4_pre_Mn = dfout[3]['absorption'] - dfout[0]['absorption']   # Mn(IV) - pre_edge

post_pre_Zn = dfout[8]['absorption'] - dfout[5]['absorption']  # post - pre_edge of Zn
ZHS_pre_Zn = dfout[6]['absorption'] - dfout[5]['absorption']   # ZHS - pre_edge
Zn2_pre_Zn = dfout[7]['absorption'] - dfout[5]['absorption']   # Zn(II) - pre_edge

norm_Mn_III_IV = Mn3_norm_Mn/Mn4_norm_Mn
norm_Mn_L_S = (2*Mn2_norm_Mn)/(Mn3_norm_Mn+Mn4_norm_Mn)
norm_Zn_ZHS_II = ZHS_norm_Zn/Zn2_norm_Zn

pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], post_pre_Mn], axis = 1).dropna().to_csv(path.join(norm_pre_edge_folder_E,'post_pre_Mn_all_E.dat'),index=None, sep= ' ')
pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], post_pre_Zn], axis = 1).dropna().to_csv(path.join(norm_pre_edge_folder_E,'post_pre_Zn_all_E.dat'),index=None, sep= ' ')
pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], norm_Mn_III_IV], axis = 1).dropna().to_csv(path.join(norm_pre_edge_folder_E,'norm_Mn_III_IV_all_E.dat'),index=None, sep= ' ')
pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], norm_Mn_L_S], axis = 1).dropna().to_csv(path.join(norm_pre_edge_folder_E,'norm_Mn_L_S_all_E.dat'),index=None, sep= ' ')
pd.concat([dfout[0][columns:=['Pt_No','tripod_x','tripod_z']], norm_Zn_ZHS_II], axis = 1).dropna().to_csv(path.join(norm_pre_edge_folder_E,'norm_Zn_ZHS_II_all_E.dat'),index=None, sep= ' ')

# 筛选出那些奇怪的点
invalidindex = post_pre_Mn.index.where((post_pre_Mn < -0.05)|(post_pre_Mn > 0.05)).dropna() # True vaule
post_pre_Mn = post_pre_Mn.iloc[post_pre_Mn.index.isin(invalidindex)]
Mn2_pre_Mn = Mn2_pre_Mn.iloc[Mn2_pre_Mn.index.isin(invalidindex)]
Mn3_pre_Mn = Mn3_pre_Mn.iloc[Mn3_pre_Mn.index.isin(invalidindex)]
Mn4_pre_Mn = Mn4_pre_Mn.iloc[Mn4_pre_Mn.index.isin(invalidindex)]

Mn_II = Mn2_pre_Mn/post_pre_Mn
Mn_III = Mn3_pre_Mn/post_pre_Mn
Mn_IV = Mn4_pre_Mn/post_pre_Mn

# Mn_III_IV = Mn3_pre_Mn/Mn4_pre_Mn
# Mn_L_S = (2*Mn2_pre_Mn)/(Mn3_pre_Mn+Mn4_pre_Mn)


invalidindex_Zn = post_pre_Zn.index.where(post_pre_Zn > 0.02).dropna() # True vaule
post_pre_Zn  = post_pre_Zn.iloc[post_pre_Zn.index.isin(invalidindex_Zn)]
ZHS_pre_Zn = ZHS_pre_Zn.iloc[ZHS_pre_Zn.index.isin(invalidindex_Zn)]
Zn2_pre_Zn = Zn2_pre_Zn.iloc[Zn2_pre_Zn.index.isin(invalidindex_Zn)]

ZHS = ZHS_pre_Zn/post_pre_Zn
Zn_II = Zn2_pre_Zn/post_pre_Zn

# Zn_ZHS_II = ZHS/Zn_II

# position
pos_Mn = dfout[0].iloc[dfout[0].index.isin(invalidindex)][columns:=['Pt_No','tripod_x','tripod_z']]
pos_Zn = dfout[5].iloc[dfout[5].index.isin(invalidindex)][columns:=['Pt_No','tripod_x','tripod_z']]

pd.concat([pos_Mn,Mn_II], axis = 1, ignore_index = False).dropna().to_csv(path.join(norm_pre_edge_folder_E,'Mn_II_all_E.dat'),index=None, sep= ' ')
pd.concat([pos_Mn,Mn_III], axis = 1, ignore_index = False).dropna().to_csv(path.join(norm_pre_edge_folder_E,'Mn_III_all_E.dat'),index=None, sep= ' ')
pd.concat([pos_Mn,Mn_IV], axis = 1, ignore_index = False).dropna().to_csv(path.join(norm_pre_edge_folder_E,'Mn_IV_all_E.dat'),index=None, sep= ' ')
pd.concat([pos_Zn,Zn_II], axis = 1, ignore_index = False).dropna().to_csv(path.join(norm_pre_edge_folder_E,'Zn_II_all_E.dat'),index=None, sep= ' ')
pd.concat([pos_Zn,ZHS], axis = 1, ignore_index = False).dropna().to_csv(path.join(norm_pre_edge_folder_E,'ZHS_all_E.dat'),index=None, sep= ' ')

# 把每一个合并的文件读取当中的每一小部分，然后画图
Datalist=glob.glob(path.join(norm_pre_edge_folder_E,file_key))

for Data_file in Datalist:
    # building a folder for each energy file
    name_folder_E =path.split(Data_file)[-1][:-10]
    path_file_folder_E = create_folders(norm_pre_edge_folder_E, name_folder_E)

    # reading in file and splitting into scans
    dfin2 = pd.read_csv(Data_file, engine = 'c', sep='\s+', index_col = None, header = 0, comment="#")
    scancuttoff_index_list=dfin2[(dfin2.loc[:,'Pt_No']==dfin2.loc[:,'Pt_No'].min())].index.tolist()
    # scancuttoff_index_list=dfin2[(dfin2.loc[:,'Pt_No']==dfin2.loc[:,'Pt_No'].max())].index.tolist()
    # print(dfin2)
    print(scancuttoff_index_list)

    # finding global min and max intensity cheapo way for transmission
    vmin = dfin2.loc[:,'absorption'].values.min()
    vmax = dfin2.loc[:,'absorption'].values.max()
    print(vmin,vmax)

    for i in range(len(scancuttoff_index_list)):
        scan_name_E= name_folder_E+'_%s'%"{:04d}".format(i+1)

        if i==0:
            a=0
            b=scancuttoff_index_list[i+1]
        else:
            a=scancuttoff_index_list[i-1]
            b=scancuttoff_index_list[i]
        # if i==0:
        #     a=0
        #     b=scancuttoff_index_list[i]  + 1
        # else:
        #     a=scancuttoff_index_list[i-1] + 1
        #     b=scancuttoff_index_list[i] + 1

        slice_energy_E = dfin2.iloc[a:b, :].pivot(index='tripod_z', columns='tripod_x', values='absorption')
        slice_energy_E.to_csv(path.join(path_file_folder_E,  scan_name_E+'.dat'), sep=' ', index=True, header=True) 
        # slice_energy_E.to_csv(path.join(path_file_folder_E,  scan_name_E+'.dat'), sep=' ', index=None, header=None)
        figure = data2map(slice_energy_E, scan_name_E, todisplay=False, vmin=vmin, vmax=vmax)
        figure.savefig(path.join(path_file_folder_E, scan_name_E+'.tif'), bbox_inches='tight', dpi=600)
        plt.close()

print('Done')